# constellation detection system

In [ ]:
import os
import yaml
from ultralytics import YOLO
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import torch
from tqdm import tqdm

# Configuration
DATASET_PATH = "stars_constellations_dataset"  # Path to dataset
IMG_SIZE = 640  # Target image size
BATCH_SIZE = 16
EPOCHS = 10
LEARNING_RATE = 0.0005
MODEL_TYPE = "yolov8s.pt"  # Using the small variant (11.2M parameters)

# Define constellation classes (16 classes as per requirements)
CLASSES = [
    "Aquila", "Bootes", "Canis Major", "Canis Minor", "Cassiopeia",
    "Cygnus", "Gemini", "Leo", "Lyra", "Moon", 
    "Orion", "Pleiades", "Sagittarius", "Taurus", "Ursa Major", "Moon"
]

def create_dataset_structure():
    """
    Create the necessary directory structure for YOLOv8 format
    """
    os.makedirs(f"{DATASET_PATH}/images/train", exist_ok=True)
    os.makedirs(f"{DATASET_PATH}/images/val", exist_ok=True)
    os.makedirs(f"{DATASET_PATH}/images/test", exist_ok=True)
    os.makedirs(f"{DATASET_PATH}/labels/train", exist_ok=True)
    os.makedirs(f"{DATASET_PATH}/labels/val", exist_ok=True)
    os.makedirs(f"{DATASET_PATH}/labels/test", exist_ok=True)
    
    return

def preprocess_images(input_dir, output_dir):
    """
    Preprocess images by resizing to 640x640 as per requirements
    """
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    image_files = [f for f in os.listdir(input_dir) if f.endswith(('.jpg', '.png', '.jpeg'))]
    
    for img_file in tqdm(image_files, desc="Preprocessing images"):
        img_path = os.path.join(input_dir, img_file)
        img = cv2.imread(img_path)
        
        # Check if image loaded correctly
        if img is None:
            print(f"Failed to load image: {img_path}")
            continue
        
        # Resize to square 640x640
        img_resized = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
        
        # Save the preprocessed image
        output_path = os.path.join(output_dir, img_file)
        cv2.imwrite(output_path, img_resized)
    
    return

def split_dataset(image_list, labels_list, train_ratio=0.7, val_ratio=0.2, test_ratio=0.1):
    """
    Split dataset into train, validation and test sets based on specified ratio
    """
    # First split into train and temporary sets
    train_images, temp_images, train_labels, temp_labels = train_test_split(
        image_list, labels_list, test_size=(val_ratio + test_ratio), random_state=42
    )
    
    # Split temporary set into validation and test sets
    val_ratio_adjusted = val_ratio / (val_ratio + test_ratio)
    val_images, test_images, val_labels, test_labels = train_test_split(
        temp_images, temp_labels, test_size=(1 - val_ratio_adjusted), random_state=42
    )
    
    return (train_images, train_labels), (val_images, val_labels), (test_images, test_labels)

def create_yaml_config():
    """
    Create YAML configuration file for YOLOv8 training
    """
    config = {
        'path': os.path.abspath(DATASET_PATH),
        'train': 'images/train',
        'val': 'images/val',
        'test': 'images/test',
        'names': {i: name for i, name in enumerate(CLASSES)}
    }
    
    with open(f"{DATASET_PATH}/constellation_data.yaml", 'w') as f:
        yaml.dump(config, f, default_flow_style=False)
    
    print(f"Created YAML configuration at {DATASET_PATH}/constellation_data.yaml")
    return f"{DATASET_PATH}/constellation_data.yaml"

def train_model(yaml_path):
    """
    Train the YOLOv8s model using the dataset
    """
    # Load a pretrained YOLOv8s model
    model = YOLO(MODEL_TYPE)
    
    # Train the model
    results = model.train(
        data=yaml_path,
        epochs=EPOCHS,
        batch=BATCH_SIZE,
        imgsz=IMG_SIZE,
        optimizer="Adam",  # As per requirements
        lr0=LEARNING_RATE,  # Initial learning rate
        conf=0.25,  # Detection confidence threshold
        name="constellation_detector",
        device=0 if torch.cuda.is_available() else 'cpu'
    )
    
    return model, results

def evaluate_model(model):
    """
    Evaluate model performance using mAP50 and other metrics
    """
    # Validate the model (this will calculate mAP50 as specified in requirements)
    results = model.val()
    
    metrics = {
        "mAP50": float(results.box.map50),  # Convert to float
        "precision": float(results.box.p),   # Convert to float
        "recall": float(results.box.r),      # Convert to float
        "f1": float(results.box.f1)          # Convert to float
    }
    
    print("\nEvaluation Metrics:")
    print(f"mAP50: {metrics['mAP50']:.4f}")
    print(f"Precision: {metrics['precision']:.4f}")
    print(f"Recall: {metrics['recall']:.4f}")
    print(f"F1 Score: {metrics['f1']:.4f}")
    
    return metrics

def detect_constellations(model, image_path):
    """
    Perform constellation detection on a single image
    """
    # Run inference
    results = model.predict(image_path, conf=0.25)
    
    # Process results
    result = results[0]
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # Display results
    plt.figure(figsize=(10, 10))
    plt.imshow(result.plot())
    plt.axis('off')
    plt.title('Constellation Detection')
    plt.show()
    
    # Print detections
    for box in result.boxes:
        class_id = int(box.cls[0].item())
        confidence = box.conf[0].item()
        coordinates = box.xyxy[0].tolist()
        print(f"Detected {CLASSES[class_id]} with confidence {confidence:.2f} at {coordinates}")
    
    return result

def main():
    # Step 1: Create dataset structure
    create_dataset_structure()
    
    # Note: At this point you would need to:
    # 1. Collect/prepare your 1,750 labeled constellation images
    # 2. Convert annotations to YOLO format
    # 3. Place images and labels in the appropriate directories
    print("Please prepare your dataset by placing images and labels in the appropriate directories")
    
    # Step 2: Create YAML configuration
    yaml_path = create_yaml_config()
    
    # Step 3: Train the model
    print("Training YOLOv8s model...")
    model, training_results = train_model(yaml_path)
    
    # Step 4: Evaluate the model
    print("Evaluating model performance...")
    metrics = evaluate_model(model)
    
    # Save the trained model
    model_save_path = "models/constellation_detector_yolov8s.pt"
    os.makedirs(os.path.dirname(model_save_path), exist_ok=True)
    model.export(format="onnx")  # Export to ONNX format
    print(f"Model saved to {model_save_path}")
    
    # For demonstration, you would use your own test image
    # detect_constellations(model, "path_to_test_image.jpg")
    
    print("Constellation detection model training and evaluation complete!")

if __name__ == "__main__":
    main()

Please prepare your dataset by placing images and labels in the appropriate directories
Created YAML configuration at stars_constellations_dataset/constellation_data.yaml
Training YOLOv8s model...
New https://pypi.org/project/ultralytics/8.3.113 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.112  Python-3.11.5 torch-2.6.0+cu118 CUDA:0 (NVIDIA GeForce GTX 1070, 8192MiB)
engine\trainer: task=detect, mode=train, model=yolov8s.pt, data=stars_constellations_dataset/constellation_data.yaml, epochs=10, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=constellation_detector5, exist_ok=False, pretrained=True, optimizer=Adam, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=0

100%|██████████| 5.35M/5.35M [00:00<00:00, 5.61MB/s]


AMP: checks passed 
train: Fast image access  (ping: 0.10.0 ms, read: 5.41.2 MB/s, size: 35.9 KB)


train: Scanning C:\Users\Dell\Downloads\Star Constellation\stars_constellations_dataset\labels\train.cache... 1530 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1530/1530 [00:00<?, ?it/s]


val: Fast image access  (ping: 0.20.1 ms, read: 3.80.6 MB/s, size: 37.7 KB)


...

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:01<00:00,  2.94it/s]


                   all        146        561      0.877      0.931      0.941      0.574
                Aquila         28         28      0.966          1      0.995      0.669
                Bootes         34         34      0.944          1      0.995      0.665
           Canis Major         29         29      0.935          1      0.995      0.644
           Canis Minor         35         35      0.886      0.886       0.93      0.419
            Cassiopeia         49         49       0.86          1      0.976      0.587
                Cygnus         35         35      0.739      0.971      0.972       0.72
                Gemini         36         36          1      0.944      0.972      0.647
                   Leo         30         30      0.882          1      0.987      0.663
                  Lyra         41         41      0.755      0.976       0.97      0.488
                  Moon         40         40      0.744        0.8      0.801      0.354
                 Orio

val: Scanning C:\Users\Dell\Downloads\Star Constellation\stars_constellations_dataset\labels\val.cache... 146 images, 0 backgrounds, 0 corrupt: 100%|██████████| 146/146 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 10/10 [00:02<00:00,  4.90it/s]


                   all        146        561      0.877      0.928       0.94      0.574
                Aquila         28         28      0.966          1      0.994      0.668
                Bootes         34         34      0.944          1      0.995      0.668
           Canis Major         29         29      0.935          1      0.995      0.644
           Canis Minor         35         35      0.886      0.886       0.93      0.416
            Cassiopeia         49         49       0.86          1      0.976      0.587
                Cygnus         35         35      0.739      0.971      0.971      0.724
                Gemini         36         36          1      0.944      0.972      0.647
                   Leo         30         30      0.882          1      0.987      0.663
                  Lyra         41         41      0.755      0.976       0.97      0.489
                  Moon         40         40      0.762        0.8      0.803      0.362
                 Orio

TypeError: unsupported format string passed to numpy.ndarray.__format__

# Constellation Detection Model Evaluation

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from ultralytics import YOLO
import cv2
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import torch
import pandas as pd

# Configuration
MODEL_PATH = "models/constellation_detector_yolov8s.pt"  # Path to trained model
TEST_DATA_PATH = "stars_constellations_dataset/images/test"  # Path to test images
CONF_THRESHOLD = 0.25  # Confidence threshold for detection
IOU_THRESHOLD = 0.5  # IoU threshold for NMS

# Constellation classes (16 classes)
CLASSES = [
    "Aquila", "Bootes", "Canis Major", "Canis Minor", "Cassiopeia",
    "Cygnus", "Gemini", "Leo", "Lyra", "Moon", 
    "Orion", "Pleiades", "Sagittarius", "Taurus", "Ursa Major", "Moon"
]

def load_model():
    """Load the trained YOLOv8s model"""
    try:
        model = YOLO(MODEL_PATH)
        print(f"Model loaded successfully from {MODEL_PATH}")
        return model
    except Exception as e:
        print(f"Error loading model: {e}")
        return None

def evaluate_on_test_set(model):
    """Evaluate the model on the test dataset"""
    if model is None:
        print("Model not loaded. Cannot evaluate.")
        return
    
    results = model.val(data=f"{os.path.dirname(TEST_DATA_PATH)}/data.yaml")
    
    # Extract metrics
    metrics = {
        "mAP50": results.box.map50,  # Primary metric
        "precision": results.box.p,   # Secondary metric
        "recall": results.box.r,      # Secondary metric
        "f1": results.box.f1          # Secondary metric (F1 Score)
    }
    
    # Display metrics
    print("\nEvaluation Metrics:")
    print(f"mAP50: {metrics['mAP50']:.4f}")
    print(f"Precision: {metrics['precision']:.4f}")
    print(f"Recall: {metrics['recall']:.4f}")
    print(f"F1 Score: {metrics['f1']:.4f}")
    
    return metrics

def visualize_results(model, num_samples=5):
    """Visualize detection results on random test samples"""
    if model is None:
        print("Model not loaded. Cannot visualize.")
        return
    
    # Get test images
    test_images = [os.path.join(TEST_DATA_PATH, f) for f in os.listdir(TEST_DATA_PATH) 
                  if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
    
    if len(test_images) == 0:
        print("No test images found.")
        return
    
    # Select random samples
    if len(test_images) > num_samples:
        test_samples = np.random.choice(test_images, num_samples, replace=False)
    else:
        test_samples = test_images
    
    # Create figure for visualization
    fig, axes = plt.subplots(1, len(test_samples), figsize=(20, 4))
    if len(test_samples) == 1:
        axes = [axes]
    
    # Process each sample
    for i, img_path in enumerate(test_samples):
        # Run inference
        results = model.predict(img_path, conf=CONF_THRESHOLD, iou=IOU_THRESHOLD)
        result = results[0]
        
        # Get the original image for display
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        # Plot result
        result_img = result.plot()
        axes[i].imshow(result_img)
        axes[i].set_title(f"Sample {i+1}")
        axes[i].axis('off')
        
        # Print detections
        detections = []
        for box in result.boxes:
            class_id = int(box.cls[0].item())
            confidence = box.conf[0].item()
            detections.append(f"{CLASSES[class_id]}: {confidence:.2f}")
        
        print(f"Sample {i+1} detections:", ", ".join(detections))
    
    plt.tight_layout()
    plt.savefig("constellation_detection_samples.png")
    plt.show()

def plot_confusion_matrix(model):
    """Plot confusion matrix for classification results"""
    if model is None:
        print("Model not loaded. Cannot plot confusion matrix.")
        return
    
    # Get all test images
    test_images = [os.path.join(TEST_DATA_PATH, f) for f in os.listdir(TEST_DATA_PATH) 
                  if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
    
    # Get ground truth labels from file names or label files
    true_labels = []
    pred_labels = []
    
    for img_path in test_images:
        # Get ground truth class (assuming filename contains class info)
        # In real scenario, you would read this from label files
        img_name = os.path.basename(img_path)
        label_path = img_path.replace('images', 'labels').replace('.jpg', '.txt').replace('.png', '.txt')
        
        if os.path.exists(label_path):
            with open(label_path, 'r') as f:
                line = f.readline().strip()
                if line:
                    class_id = int(line.split(' ')[0])
                    true_labels.append(class_id)
                else:
                    continue
        else:
            continue
        
        # Run inference
        results = model.predict(img_path, conf=CONF_THRESHOLD)
        result = results[0]
        
        # Get highest confidence detection
        if len(result.boxes) > 0:
            # Find detection with highest confidence
            confidences = [box.conf[0].item() for box in result.boxes]
            max_conf_idx = np.argmax(confidences)
            pred_class_id = int(result.boxes[max_conf_idx].cls[0].item())
            pred_labels.append(pred_class_id)
        else:
            # No detection, skip this image
            true_labels.pop()  # Remove the corresponding true label
    
    # Create confusion matrix
    if len(true_labels) > 0 and len(pred_labels) > 0:
        cm = confusion_matrix(true_labels, pred_labels, labels=range(len(CLASSES)))
        
        # Plot
        plt.figure(figsize=(12, 10))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=CLASSES, yticklabels=CLASSES)
        plt.xlabel('Predicted')
        plt.ylabel('True')
        plt.title('Confusion Matrix for Constellation Detection')
        plt.savefig("confusion_matrix.png")
        plt.show()
        
        # Classification report
        report = classification_report(true_labels, pred_labels, target_names=CLASSES)
        print("\nClassification Report:")
        print(report)
    else:
        print("Not enough data to create confusion matrix.")

def visualize_loss_curves():
    """Visualize training loss curves from the results file"""
    try:
        # Location of results CSV (this path might vary based on your training output)
        results_path = "runs/detect/constellation_detector/results.csv"
        
        if not os.path.exists(results_path):
            print(f"Results file not found at {results_path}")
            return
        
        # Load results
        results = pd.read_csv(results_path)
        
        # Plot loss curves
        plt.figure(figsize=(15, 10))
        
        # Plot box loss
        plt.subplot(3, 1, 1)
        plt.plot(results['epoch'], results['box_loss'], label='Box Loss', color='blue')
        plt.title('Box Loss')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.grid(True)
        plt.legend()
        
        # Plot classification loss
        plt.subplot(3, 1, 2)
        plt.plot(results['epoch'], results['cls_loss'], label='Classification Loss', color='green')
        plt.title('Classification Loss')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.grid(True)
        plt.legend()
        
        # Plot distribution focal loss (DFL)
        plt.subplot(3, 1, 3)
        plt.plot(results['epoch'], results['dfl_loss'], label='Distribution Focal Loss', color='red')
        plt.title('Distribution Focal Loss (DFL)')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.grid(True)
        plt.legend()
        
        plt.tight_layout()
        plt.savefig("training_loss_curves.png")
        plt.show()
    except Exception as e:
        print(f"Error plotting loss curves: {e}")

def main():
    # Step 1: Load model
    model = load_model()
    
    # Step 2: Evaluate on test set
    metrics = evaluate_on_test_set(model)
    
    # Step 3: Visualize results on sample images
    visualize_results(model, num_samples=5)
    
    # Step 4: Plot confusion matrix
    plot_confusion_matrix(model)
    
    # Step 5: Visualize loss curves
    visualize_loss_curves()
    
    print("Evaluation complete!")

if __name__ == "__main__":
    main()

# Constellation Detection Inference Demo

In [ ]:
import os
import cv2
import numpy as np
from ultralytics import YOLO
import matplotlib.pyplot as plt
import argparse
from PIL import Image, ImageDraw, ImageFont
import time

# Configuration
MODEL_PATH = "models/constellation_detector_yolov8s.pt"  # Path to trained model
CONF_THRESHOLD = 0.25  # Confidence threshold for detection
IOU_THRESHOLD = 0.5    # IoU threshold for non-maximum suppression
CLASSES = [
    "Aquila", "Bootes", "Canis Major", "Canis Minor", "Cassiopeia",
    "Cygnus", "Gemini", "Leo", "Lyra", "Moon", 
    "Orion", "Pleiades", "Sagittarius", "Taurus", "Ursa Major", "Moon"
]

def load_model():
    """Load the YOLOv8s model for inference"""
    try:
        model = YOLO(MODEL_PATH)
        print(f"Model loaded successfully from {MODEL_PATH}")
        return model
    except Exception as e:
        print(f"Error loading model: {e}")
        return None

def process_image(model, image_path):
    """Process a single image and display results"""
    if model is None:
        print("Model not loaded. Cannot process image.")
        return None
    
    # Load image
    try:
        image = cv2.imread(image_path)
        if image is None:
            print(f"Failed to load image: {image_path}")
            return None
        
        # Convert to RGB for display
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        # Run inference
        start_time = time.time()
        results = model.predict(image_path, conf=CONF_THRESHOLD, iou=IOU_THRESHOLD)
        inference_time = time.time() - start_time
        
        result = results[0]
        
        # Create result image
        result_img = result.plot()
        
        # Print detection information
        print(f"\nProcessed image: {image_path}")
        print(f"Inference time: {inference_time:.4f} seconds")
        print("Detections:")
        
        for i, box in enumerate(result.boxes):
            class_id = int(box.cls[0].item())
            confidence = box.conf[0].item()
            coordinates = box.xyxy[0].tolist()  # x1, y1, x2, y2 format
            
            print(f"  {i+1}. {CLASSES[class_id]}: {confidence:.4f} at {[round(c, 2) for c in coordinates]}")
        
        return result_img
    
    except Exception as e:
        print(f"Error processing image: {e}")
        return None

def process_directory(model, directory_path, output_dir="output"):
    """Process all images in a directory and save results"""

testing cuda installation

In [1]:
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"GPU device name: {torch.cuda.get_device_name(0)}")

c:\Users\Dell\anaconda3\Lib\site-packages\torch\utils\_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(


PyTorch version: 2.6.0+cu118
CUDA available: True
CUDA version: 11.8
GPU device name: NVIDIA GeForce GTX 1070
